In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('survey.csv')  # Replace with your dataset's filename
print(df.head())

             Timestamp  Age  Gender         Country state self_employed  \
0  2014-08-27 11:29:31   37  Female   United States    IL           NaN   
1  2014-08-27 11:29:37   44       M   United States    IN           NaN   
2  2014-08-27 11:29:44   32    Male          Canada   NaN           NaN   
3  2014-08-27 11:29:46   31    Male  United Kingdom   NaN           NaN   
4  2014-08-27 11:30:22   31    Male   United States    TX           NaN   

  family_history treatment work_interfere    no_employees  ...  \
0             No       Yes          Often            6-25  ...   
1             No        No         Rarely  More than 1000  ...   
2             No        No         Rarely            6-25  ...   
3            Yes       Yes          Often          26-100  ...   
4             No        No          Never         100-500  ...   

                leave mental_health_consequence phys_health_consequence  \
0       Somewhat easy                        No                      No   
1 

In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1259 entries, 0 to 1258
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Timestamp                  1259 non-null   object
 1   Age                        1259 non-null   int64 
 2   Gender                     1259 non-null   object
 3   Country                    1259 non-null   object
 4   state                      744 non-null    object
 5   self_employed              1241 non-null   object
 6   family_history             1259 non-null   object
 7   treatment                  1259 non-null   object
 8   work_interfere             995 non-null    object
 9   no_employees               1259 non-null   object
 10  remote_work                1259 non-null   object
 11  tech_company               1259 non-null   object
 12  benefits                   1259 non-null   object
 13  care_options               1259 non-null   object
 14  wellness

In [ ]:
print(df.isnull().sum())

Timestamp                       0
Age                             0
Gender                          0
Country                         0
state                         515
self_employed                  18
family_history                  0
treatment                       0
work_interfere                264
no_employees                    0
remote_work                     0
tech_company                    0
benefits                        0
care_options                    0
wellness_program                0
seek_help                       0
anonymity                       0
leave                           0
mental_health_consequence       0
phys_health_consequence         0
coworkers                       0
supervisor                      0
mental_health_interview         0
phys_health_interview           0
mental_vs_physical              0
obs_consequence                 0
comments                     1095
dtype: int64


In [ ]:
# Filter for women
df_women = df[df['Gender'].str.contains('Female', case=False, na=False)]
print(df_women.head())

              Timestamp  Age  Gender        Country state self_employed  \
0   2014-08-27 11:29:31   37  Female  United States    IL           NaN   
6   2014-08-27 11:31:50   35  Female  United States    MI           NaN   
8   2014-08-27 11:32:39   42  Female  United States    IL           NaN   
12  2014-08-27 11:33:23   42  female  United States    CA           NaN   
15  2014-08-27 11:34:00   29  female  United States    IL           NaN   

   family_history treatment work_interfere no_employees  ...  \
0              No       Yes          Often         6-25  ...   
6             Yes       Yes      Sometimes          1-5  ...   
8             Yes       Yes      Sometimes      100-500  ...   
12            Yes       Yes      Sometimes       26-100  ...   
15            Yes       Yes         Rarely       26-100  ...   

                 leave mental_health_consequence phys_health_consequence  \
0        Somewhat easy                        No                      No   
6   Somewhat

In [ ]:
print(df_women.isnull().sum())

Timestamp                      0
Age                            0
Gender                         0
Country                        0
state                         59
self_employed                  5
family_history                 0
treatment                      0
work_interfere                24
no_employees                   0
remote_work                    0
tech_company                   0
benefits                       0
care_options                   0
wellness_program               0
seek_help                      0
anonymity                      0
leave                          0
mental_health_consequence      0
phys_health_consequence        0
coworkers                      0
supervisor                     0
mental_health_interview        0
phys_health_interview          0
mental_vs_physical             0
obs_consequence                0
comments                     158
dtype: int64


In [ ]:
features = ['Age', 'family_history', 'work_interfere', 'benefits', 'seek_help', 'anonymity', 'mental_health_consequence']
target = 'treatment'
df_women = df_women[features + [target]]

In [ ]:
df_women = df_women.dropna()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Encode categorical features
encoder = OneHotEncoder(drop='first')  # Remove the 'sparse' parameter
encoded_features = encoder.fit_transform(df_women[['family_history', 'work_interfere', 'benefits', 'seek_help', 'anonymity', 'mental_health_consequence']])
encoded_df = pd.DataFrame(encoded_features.toarray(), columns=encoder.get_feature_names_out())  # Use .toarray() to convert to a dense array

In [ ]:
# Step 1: Filter the Dataset for Women
df_women = df[df['Gender'].str.contains('Female', case=False, na=False)]
print(df_women.head())

# Step 2: Select Relevant Features
features = ['Age', 'family_history', 'work_interfere', 'benefits', 'seek_help', 'anonymity', 'mental_health_consequence']
target = 'treatment'
df_women = df_women[features + [target]]
df_women = df_women.dropna()

# Step 3: Preprocess the Data
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first')  # Remove the 'sparse' parameter
encoded_features = encoder.fit_transform(df_women[['family_history', 'work_interfere', 'benefits', 'seek_help', 'anonymity', 'mental_health_consequence']])
encoded_df = pd.DataFrame(encoded_features.toarray(), columns=encoder.get_feature_names_out())  # Use .toarray()

X = pd.concat([df_women[['Age']], encoded_df], axis=1)
y = df_women[target]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Train the Model
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Step 5: Evaluate the Model
from sklearn.metrics import classification_report, accuracy_score
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Step 6: Save the Model
import joblib
joblib.dump(model, 'women_mental_health_model.pkl')
from google.colab import files
files.download('women_mental_health_model.pkl')

# Step 7: Make Predictions
model = joblib.load('women_mental_health_model.pkl')
example_input = pd.DataFrame({
    'Age': [30],
    'family_history_Yes': [1],
    'work_interfere_Sometimes': [1],
    'benefits_Yes': [1],
    'seek_help_Yes': [1],
    'anonymity_Yes': [1],
    'mental_health_consequence_No': [1]
})
prediction = model.predict(example_input)
print("Prediction:", prediction)

              Timestamp  Age  Gender        Country state self_employed  \
0   2014-08-27 11:29:31   37  Female  United States    IL           NaN   
6   2014-08-27 11:31:50   35  Female  United States    MI           NaN   
8   2014-08-27 11:32:39   42  Female  United States    IL           NaN   
12  2014-08-27 11:33:23   42  female  United States    CA           NaN   
15  2014-08-27 11:34:00   29  female  United States    IL           NaN   

   family_history treatment work_interfere no_employees  ...  \
0              No       Yes          Often         6-25  ...   
6             Yes       Yes      Sometimes          1-5  ...   
8             Yes       Yes      Sometimes      100-500  ...   
12            Yes       Yes      Sometimes       26-100  ...   
15            Yes       Yes         Rarely       26-100  ...   

                 leave mental_health_consequence phys_health_consequence  \
0        Somewhat easy                        No                      No   
6   Somewhat

ValueError: Found input variables with inconsistent numbers of samples: [319, 167]

In [ ]:
print(X.isnull().sum())
print(y.isnull().sum())

Age                              152
family_history_Yes               152
work_interfere_Often             152
work_interfere_Rarely            152
work_interfere_Sometimes         152
benefits_No                      152
benefits_Yes                     152
seek_help_No                     152
seek_help_Yes                    152
anonymity_No                     152
anonymity_Yes                    152
mental_health_consequence_No     152
mental_health_consequence_Yes    152
dtype: int64
0


In [ ]:
X = X.dropna()
y = y[X.index]  # Ensure y aligns with X after dropping rows

In [ ]:
print(X.shape, y.shape)

(15, 13) (15,)


In [ ]:
y = y[X.index]

In [ ]:
# Step 1: Split the Data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Step 2: Train the Model
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Step 3: Evaluate the Model
from sklearn.metrics import classification_report, accuracy_score
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Step 4: Save the Model
import joblib
joblib.dump(model, 'women_mental_health_model.pkl')
from google.colab import files
files.download('women_mental_health_model.pkl')

# Step 5: Make Predictions
example_input = pd.DataFrame({
    'Age': [30],
    'family_history_Yes': [1],
    'work_interfere_Often': [0],
    'work_interfere_Rarely': [0],
    'work_interfere_Sometimes': [1],
    'benefits_No': [0],
    'benefits_Yes': [1],
    'seek_help_No': [0],
    'seek_help_Yes': [1],
    'anonymity_No': [0],
    'anonymity_Yes': [1],
    'mental_health_consequence_No': [1],
    'mental_health_consequence_Yes': [0]
})
prediction = model.predict(example_input)
print("Prediction:", prediction)

(12, 13) (3, 13) (12,) (3,)
Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

         Yes       1.00      1.00      1.00         3

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Prediction: ['Yes']
